In [1]:
import requests
import datetime
import pandas as pd

In [2]:
#name="Paola"
#name="Denis"
#name="Stéphane"
#name="Gwang-Hi"
#name="Jean-Marc"
#name="Yannick"

In [3]:
if name=="Paola":
    halid="paola-nava"
    firstname="Paola"
    lastname="NAVA"
elif name=="Denis":
    halid="denis-hagebaum-reignier"
    firstname="Denis"
    lastname="HAGEBAUM-REIGNIER"
elif name=="Stéphane":
    halid="stephane-humbel"
    firstname="Stéphane"
    lastname="HUMBEL"
elif name=="Gwang-Hi":
    halid="gwang-hi-jeung"
    firstname="Gwang-Hi"
    lastname="JEUNG"
elif name=="Jean-Marc":
    halid="jean-marc-mattalia"
    firstname="Jean-Marc"
    lastname="MATTALIA"
elif name=="Yannick":
    halid="yannickcarissan"
    firstname="Yannick"
    lastname="CARISSAN"

In [4]:
now = datetime.datetime.now()
this_year = now.year

r=requests.get("https://api.archives-ouvertes.fr/search//?q=(authIdHal_s:{}%20OR%20authFullName_sci:%22{}%20{}%22)%20AND%20docType_s:ART&rows=10000&sort=producedDate_tdate%20desc&fl=docid,docType_s,authFullName_s,authAlphaLastNameFirstNameId_fs,title_s,number_s,producedDate_s,producedDateY_i,files_s,label_s,label_xml,halId_s,doiId_s".format(halid,firstname,lastname))

response_dec = r.json()

number_of_docs = response_dec["response"]["numFound"]
docs = response_dec["response"]["docs"]
print("Number of documents : {}".format(number_of_docs))

df = pd.DataFrame()
for doc in docs:
    doctype = doc["docType_s"]
    docid = doc["docid"]
    halid = doc["halId_s"]
    title = doc["title_s"][0]
    authors_full_name=""
    for auth in doc["authFullName_s"]:
        if len(authors_full_name)>0:
            authors_full_name = authors_full_name + ", " + auth
        else:
            authors_full_name = auth

    date = doc["producedDate_s"]
    year = doc["producedDateY_i"]
    try:
        doiid = doc["doiId_s"]
    except:
        doiid = "n/a"
    try:
        file_attached = doc["files_s"][0]
    except:
        file_attached = None
    if (file_attached!=None):
        hal_ok = "OK" #"\U00002714"
    else:
        hal_ok = "MISSING" #\U00002718"
    dict_doc = {}
    dict_doc["doctype"] = doctype
    dict_doc["docid"]   = docid
    dict_doc["halid"]   = halid
    dict_doc["title"]   = title
    dict_doc["authors"] = authors_full_name
    dict_doc["date"]    = date
    dict_doc["year"]    = year
    dict_doc["doiid"]   = doiid
    dict_doc["hal_ok"]  = hal_ok
    df = df.append(dict_doc, ignore_index=True)
df.index=[i for i in range(1,number_of_docs+1)]

Number of documents : 12


In [5]:
df[["authors", "title", "year", "doiid", "halid", "hal_ok"]]

,authors,title,year,doiid,halid,hal_ok
1,"Jean-Marc Mattalia, Paola Nava",C-C Bond Breaking in Addition-Elimination Reac...,2019.0,10.1002/ejoc.201900119,hal-02125621,OK
2,Jean-Marc Mattalia,The reductive decyanation reaction: an overvie...,2017.0,10.3762/bjoc.13.30,hal-01480621,OK
3,"Jean-Marc Mattalia, Paola Nava",A Computational Study of the Intramolec­ular C...,2016.0,10.1002/ejoc.201501208,hal-01450750,MISSING
4,"Jean-Marc Mattalia, Paola Nava","Gold-catalyzed cycloisomerizations of 1,6-enyn...",2014.0,10.1016/j.jorganchem.2013.09.041,hal-01450759,MISSING
5,Jean-Marc Mattalia,Preparation and chemical reactivity of 2-chrom...,2012.0,n/a,hal-01988815,MISSING
6,"C. Reynaud, Hassan Hazimeh, Jean-Marc Mattalia...",Clear-cut difference in the rearrangement of 1...,2011.0,n/a,hal-01062715,MISSING
7,"Hassan Hazimeh, Jean-Marc Mattalia, Caroline M...",Structural Effects in Radical Clocks and Mecha...,2009.0,n/a,hal-00389351,MISSING
8,"Hassan Hazimeh, Frederic Kanoufi, Catherine Co...","Radical Clocks, Solvated Electrons, and Magnes...",2008.0,n/a,hal-00378064,MISSING
9,"Hassan Hazimeh, Jean-Marc Mattalia, Mireille A...",Grignard reagent and Green chemistry : Mechani...,2006.0,n/a,hal-00378053,MISSING
10,"Jean-Marc Mattalia, Caroline Marchi Delapierre...",The reductive decyanation reaction : chemical ...,2006.0,n/a,hal-00378046,MISSING


In [6]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    
    if val == "\U00002714":
        color = "green"
    elif val == "\U00002718":
        color = "red"
    else:
        color="black"
    return 'color: %s' % color

In [7]:
s = df.style.applymap(color_negative_red, subset=['hal_ok'])